In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import plotly.express as py
import plotly.graph_objects as go
import plotly.offline as offline



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

df = pd.read_csv('../input/pakistans-largest-ecommerce-dataset/Pakistan Largest Ecommerce Dataset.csv')


In [ ]:
print(df.head())

In [ ]:
df.info()

In [ ]:
df.shape

df[['Unnamed: 21','Unnamed: 22']]

#Dropping the empty columns

df.drop(columns=['Unnamed: 21','Unnamed: 22','Unnamed: 23','Unnamed: 24','Unnamed: 25'],inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df['category_name_1'].unique()

In [ ]:
df[df['category_name_1'] == '\\N']

In [ ]:
#Replacing category types \\N with 'Not available'
df['cateogry_name'] = df['category_name_1'].replace(['Others'],"Others_1")

In [ ]:
df['category_name_1'].unique()

In [ ]:
# 1) Renaming some column names for better accessibility
# 2) Seperating columns to answer the question of best category
# 3) Will answer best category by a) Orders, b) Grand Total c) After considering order status

In [ ]:
df.columns

In [ ]:
#renaming columns for better accessibility

In [ ]:
df.rename(columns={'category_name_1':"category_name"," MV":"MV"},inplace=True)

In [ ]:
#Seperating columns for answering the question about best category

df1=df[['category_name','status','price','qty_ordered','grand_total','Year','Month']]

In [ ]:
df1[int(df1['Year'])]

In [ ]:
# Observing the different columns shows us that we have a ) nan values and a values showing up as \\N in alot of columns


df1['category_name'].unique()
df1['status'].unique()

In [ ]:
#Checking the number of values in "category_name" with"\\N" value


df1[df1['category_name']=="\\N"].count()

In [ ]:
#Replacing "\\N" with "NA"

df1.category_name.replace(['\\N'],'NA',inplace=True)
df1.status.replace(['\\N'],'NA',inplace=True)

In [ ]:
# \\N replaced with 'NA'
df1.category_name.unique()
df1.status.unique()

In [ ]:
#Checking for null values 

df1.isnull().sum()

In [ ]:
#dropping na values

df1.dropna(how='all',inplace=True)


In [ ]:
df1.isnull().sum()

# **A considerable reduction in the Dataset is observed because out of approximately 1M entries 400,000 were empty rows. Removing cancelled ordered from the data set**

In [ ]:
df2=df1[df1.status!='canceled']


# **Pivoting data on the basis of product category with grand_total in focus**

In [ ]:
df3=df2.pivot_table(index='category_name',values=['grand_total'],aggfunc=np.sum).nlargest(10,'grand_total')

In [ ]:
df3.reset_index(inplace=True)

In [ ]:
df3

In [ ]:
# Best category by Grand Total
Bar_chart1 =py.bar(df3,x='category_name',y='grand_total')
Bar_chart1.show()

# **Pivoting data on the basis of product category with quantity ordered in focus**

In [ ]:
df3=df2.pivot_table(index='category_name',values='qty_ordered',aggfunc=np.sum).nlargest(10,'qty_ordered')

In [ ]:
df3.reset_index(inplace=True)

In [ ]:
df3

In [ ]:
Bar_chart2=py.bar(df3,x='category_name',y='qty_ordered')
Bar_chart2.show()

# **We can see that the top categories based on the number of items is different where the most sold items by volume are Mens clothing rather than Mobile phones. The primary reason behind this is that the average per unit price of a Mobile is considerably higher than the average per unit price of a clothing item **

# **Would be interesting to see if this data set and this hierarchy or Top N categories for grand total and Qty Ordered holds true on an year to year basis. We have three unique years beint 2016,17 and 18**

In [ ]:
df4=df2.pivot_table(index=['Year','category_name'],values='grand_total',aggfunc=np.sum)

In [ ]:
df4

In [ ]:
df4.reset_index(inplace=True)

In [ ]:
df4

In [ ]:
offline.init_notebook_mode()


offline.iplot(
    [go.Bar(x=[df4['Year'],
               df4['category_name']],
            y=df['grand_total'])
    ]
)

# **We can see that even over the period of 3 years Mobiles & Tablets category is a consistent winner when it comes to the Grand Total (Rupees spent per cateogory). Although the volume of online purchase of Mobiles & Tablets has greatly increased from 2016 to 2017 but then sees a significant dip in 2018**

In [ ]:
df5=df2.pivot_table(index=['Year','category_name'],values='qty_ordered',aggfunc=np.sum)

In [ ]:
df5.reset_index(inplace=True)

In [ ]:
offline.iplot([go.Bar(x=[df5['Year'],df4['category_name']], y=df5['qty_ordered'])])

# **Looking at the data from a Quantity Ordered perspective we actually find out the Men's Fashion out paces Mobiles and Tablets in the first two years of the data but Mobiles & Tablets outdo Men's Fashion in year 2018. Interestingly the "Others Category is the largest category by volume in 2018. Perhaps the ecommerce industry cna shed better light on what led to the scenario where so many Misc Products were bundled into the Others category.**